In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Cleaning & Imputation/Output')
cur_date = "042023"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(mRNA_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


In [3]:
# identifying the number of replicates for each sample condition
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count, mRNA, Subject_No, Condensate, Burn_Condition, Concentration, Time_Point) 

head(count_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Duplicate_Count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [4]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count,n
<int>,<int>
6,6
5,80
4,328
3,96
2,1340
1,2361


In [5]:
# creating a new column for replicate number
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Replicate
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,1
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,2
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,3
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,4
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,5
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


Removing variables and ids with less than 25% of data. However, ids will be removed after casting the data to a wide format and back to long.

In [10]:
# # only keeping variables with at least 25% of data 
# variable_presence_df = mRNA_df %>% 
#     # creating 2 count cols: 1st for non-missing data, 2nd for all data
#     mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) #%>%
# #     group_by(Condensate, Burn_Condition, Concentration, Time_Point, mRNA) %>%
# #     summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
# #     arrange(Variable_Presence_Percent)

# head(variable_presence_df)

In [11]:
# mRNA_df %>%
#     filter(Condensate == "PBS", Burn_Condition == "PBS", Time_Point == 24, mRNA == "HMOX1")

In [6]:
# pivoting longer but first creating sample ids
missing_data_mRNA_df = mRNA_df %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(mRNA_df)[c(4:7,10)])) %>%
    select(-c("Subject_No", "ddCT")) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "ddCT_pslog2") %>%
    pivot_longer(cols = 3:8, names_to = "Subject_ID", values_to = "ddCT_pslog2") %>%
    separate(Sample_ID, c("Condensate", "Burn_Condition", "Concentration", "Time_Point", "Replicate"))

head(missing_data_mRNA_df)

mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Replicate,Subject_ID,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
HMOX1,PBS,PBS,NA,24,1,M_6,4.892803
HMOX1,PBS,PBS,NA,24,1,F_5,NA
HMOX1,PBS,PBS,NA,24,1,F_1,NA
HMOX1,PBS,PBS,NA,24,1,M_2,NA
HMOX1,PBS,PBS,NA,24,1,M_3,NA
HMOX1,PBS,PBS,NA,24,1,F_4,NA


In [41]:
# # only keeping ids with at least 25% of data 
# subject_presence_df = missing_data_mRNA_df %>% 
#     # creating 2 count cols: 1st for non-missing data, 2nd for all data
#     mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) #%>%
# #     group_by(Condensate, Burn_Condition, Concentration, Time_Point, Replicate, Subject_ID) %>%
# #     summarize(Subject_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
# #     arrange(Subject_Presence_Percent)

# # head(subject_presence_df) 

# # # getting the proteins in the entire set that will be removed
# # removed_subjects_df = subject_presence_df %>%
# #     filter(Subject_Presence_Percent < 50)

# # # getting number of sample conditions to be removed
# # dim(removed_subjects_df)[1]

In [44]:
# subject_presence_df %>%
#     filter(Condensate == "PBS", Burn_Condition == "PBS", Time_Point == 24, Subject_ID == "M_6", Replicate == 1) #%>%
# #     ungroup() %>%
# #     group_by(Subject_ID) %>%
# #     summarize(sum(data_count)/ sum(all_count))

mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Replicate,Subject_ID,ddCT_pslog2,data_count,all_count
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
HMOX1,PBS,PBS,NA,24,1,M_6,4.892803,1,1
ALDH1A3,PBS,PBS,NA,24,1,M_6,4.899654,1,1
ALDH3A1,PBS,PBS,NA,24,1,M_6,4.837012,1,1
CXCL1,PBS,PBS,NA,24,1,M_6,4.822372,1,1
CXCR1,PBS,PBS,NA,24,1,M_6,3.940873,1,1
CYP1A1,PBS,PBS,NA,24,1,M_6,5.050207,1,1
CYP1B1,PBS,PBS,NA,24,1,M_6,4.908724,1,1
GCLC,PBS,PBS,NA,24,1,M_6,4.863186,1,1
GCLM,PBS,PBS,NA,24,1,M_6,4.834127,1,1


Therefore, 420 sample conditions will be removed. 

In [34]:
# # removing those 420 sample conditions
# filtered_mRNA_df = anti_join(missing_data_mRNA_df, removed_subjects_df)

# head(filtered_mRNA_df)

Joining, by = c("Condensate", "Burn_Condition", "Concentration", "Time_Point",
"Replicate", "Subject_ID")


mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Replicate,Subject_ID,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
HMOX1,PBS,PBS,NA,24,1,M_6,4.892803
HMOX1,PBS,PBS,NA,24,1,F_5,NA
HMOX1,PBS,PBS,NA,24,1,F_1,NA
HMOX1,PBS,PBS,NA,24,1,M_2,NA
HMOX1,PBS,PBS,NA,24,1,M_3,NA
HMOX1,PBS,PBS,NA,24,1,F_4,NA


In [11]:
# converting to a wider df, making values binary (if the value is missing 0, if the value is present 1)
wider_missing_data_mRNA_df = missing_data_mRNA_df %>%
    # creating sample ids
    unite("SampleID_SubjectID", c(colnames(mRNA_df)[c(4:7,10,2)])) %>%
    pivot_wider(names_from = "mRNA", values_from = "ddCT_pslog2") %>%
    column_to_rownames(var = "SampleID_SubjectID")

head(wider_missing_data_mRNA_df)


binary_mRNA_df = wider_missing_data_mRNA_df %>%
    mutate_all(~replace(.,!is.na(.), 1)) %>%
    mutate_all(~replace(., is.na(.), 0))

head(binary_mRNA_df)

,HMOX1,ALDH3A1,CXCL1,CXCR1,GCLC,GCLM,GMCSF2,IL1B,IL6,IL8,MCP1,MUC5AC,MUC5B,NQO1,TBP,TXNRD1,ALDH1A3,CYP1A1,CYP1B1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PBS_PBS_NA_24_1_M_6,4.892803,4.837012,4.822372,3.940873,4.863186,4.834127,4.471997,4.846304,4.148413,4.787056,4.234212,4.835085,4.979284,4.840225,4.861164,4.883556,4.899654,5.050207,4.908724
PBS_PBS_NA_24_1_F_5,NA,4.826226,4.855112,3.964669,4.871636,4.840402,4.525502,4.833157,4.120831,4.798890,4.079356,4.851623,5.068697,4.836307,4.863756,4.901605,4.908853,5.040497,4.901449
PBS_PBS_NA_24_1_F_1,NA,4.790564,4.802618,3.956375,4.851443,4.822743,4.582481,4.820877,4.244659,4.838274,4.379597,5.012320,4.900861,4.784756,4.845427,4.862396,4.888831,4.979335,4.894734
PBS_PBS_NA_24_1_M_2,NA,4.815891,4.843155,3.880202,4.858772,4.829920,4.568797,4.844923,4.294227,4.822151,4.344537,4.930513,4.889592,4.819148,4.842778,4.849086,4.896734,5.051720,4.927327
PBS_PBS_NA_24_1_M_3,NA,4.783329,4.883960,3.981948,4.865541,4.836289,4.562426,4.821139,4.240050,4.843051,4.275184,4.839117,5.092434,4.822620,4.863954,4.909398,4.873411,5.104235,4.955168
PBS_PBS_NA_24_1_F_4,NA,4.821705,4.839909,3.919197,4.862229,4.837137,4.462491,4.858182,4.191909,4.763427,4.133940,4.901020,4.942372,4.831467,4.832835,4.845209,4.894228,5.120224,4.947159


,HMOX1,ALDH3A1,CXCL1,CXCR1,GCLC,GCLM,GMCSF2,IL1B,IL6,IL8,MCP1,MUC5AC,MUC5B,NQO1,TBP,TXNRD1,ALDH1A3,CYP1A1,CYP1B1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PBS_PBS_NA_24_1_M_6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PBS_PBS_NA_24_1_F_5,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PBS_PBS_NA_24_1_F_1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PBS_PBS_NA_24_1_M_2,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PBS_PBS_NA_24_1_M_3,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
PBS_PBS_NA_24_1_F_4,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [15]:
install.packages("janitor")
library(janitor)

also installing the dependency ‘snakecase’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [16]:
# identify variables that have at least 25% of samples
binary_mRNA_df %>%
    adorn_totals(where = "col",... = colnames(binary_mRNA_df)) %>% 
    filter(Total >= ncol(binary_mRNA_df) * 0.25)

,HMOX1,ALDH3A1,CXCL1,CXCR1,GCLC,GCLM,GMCSF2,IL1B,IL6,IL8,MCP1,MUC5AC,MUC5B,NQO1,TBP,TXNRD1,ALDH1A3,CYP1A1,CYP1B1,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PBS_PBS_NA_24_1_M_6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19
PBS_PBS_NA_24_1_F_5,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,18
PBS_PBS_NA_24_1_F_1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,18
PBS_PBS_NA_24_1_M_2,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,18
PBS_PBS_NA_24_1_M_3,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,18
PBS_PBS_NA_24_1_F_4,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,18
PBS_PBS_NA_4_1_M_6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19
PBS_PBS_NA_4_1_F_5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19
PBS_PBS_NA_4_1_F_1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19


# QRILC Imputation